# XGBoost

## 概念
XGBoost全名叫**eXtreme Gradient boosting**，**极端梯度提升树**，既可以用于分类也可以用于回归问题中。

## 从Boosting说起
Boosting是一族可将若学习器提升为强学习器的算法。这族算法的工作机制类似：先从初始训练集训练出**一个基学习器**，再根据基学习器的表现**对训练样本分布**进行调整，使得先前基学习器做错的训练样本在后续受到更多关注，然后基于调整后的样本分布来训练下一个基学习器；如此重复进行，直至基学习器数目达到事先指定的值T，最终将这T个基学习器进行加权结合。

以决策树为基学习器举个例子，Boosting集成学习是由多个相关联的决策树联合决策。与之对比的是随机森林算法，各个决策树是独立的、每个决策树在样本堆里随机选一批样本，随机选一批特征进行独立训练，各个决策树之间没有什么关系。

## 集成思路
XGBoost是以CART树进行组合的，先来看下CART树。如下图，通过输入用户年龄、性别进行判断用户是否喜欢玩游戏的得分值。由此得到一棵CART树模型。
    
![image.png](./pic/1.png)

对于单个的决策树模型容易出现过拟合，所以出现了集成学习方法。如下图，通过两棵树组合进行玩游戏得分值预测。其中tree1中对小男生的预测分值为2。tree2对小男生的预测分值为0.9。则该男生的最后得分值为2.9。
    
![image.png](./pic/2.png)

将上面集成学习方法推广到一般情况，可知其预测模型为：
    
![image.png](./pic/3.png)

其中K为树的总个数，fk表示第k棵树。损失函数为：
    
![image.png](./pic/4.png)

## XGBoost思路

### 核心思想是**贪心算法+最优化（二次最优化）**

先从集成算法的目标函数看起：

![image.png](./pic/5.png)

其中后面一项是正则项，为了防止模型**过拟合**，第一项是决策树中的叶子结点树，第二项是节点数值的大小。这样设置是为了决策树中叶子结点尽量少，节点数值尽量不极端。公式如下：

![image.png](./pic/6.png)

需要求解和优化的就是每个叶子结点的得分值，也就是f(x)的值。

现在对正则项进行分析，单从一棵树开始考虑。对于每一棵回归树，其模型可以写成：

![image.png](./pic/9.png)

其中w是叶子节点的得分值，q(x)表示样本x对应的叶子节点。T为该树的叶子节点个数。

在这里，可以将该树的复杂度写成：

![image.png](./pic/10.png)

复杂度计算例子如下：

![image.png](./pic/11.png)

XGBoost并不是简单重复的将几个CART树进行组合。它是一种**加法模型**，将模型上一次预测（由t-1棵树组合而成的模型）**产生的误差**作为参考进行下一棵树（第t棵树）的建立。以此，每加入一棵树，将其损失函数不断降低。因此这个目标函数不能通过传统梯度下降进行优化，而要通过加式训练来解决。

对于加法策略可以表示如下：

![image.png](./pic/7.png)

每次往模型中加入一棵树，其损失函数便会发生变化。另外在加入第t棵树时，则前面t-1棵树已经训练完成，此时前面t-1棵树的正则项和训练误差都成**已知常数项**。

如果损失函数采用均方误差时，其目标损失函数变为(这里的公式有一个地方写错了，研一的看下是哪里)：

![image.png](./pic/8.png)

现在我们只需要找到f(t)来优化上式目标。

![image.png](./pic/12.png)

在推导之前，先介绍下**泰勒展开式**：

![image.png](./pic/13.png)

这里用泰勒展开式来近似原来的目标函数，将$f_t(x_i)$看作$\Delta x$。则原目标函数可以写成：

![image.png](./pic/16.png)

令$g_i=\delta _{\hat y^{(t-1)}} \cdot l(y_i, \hat y^{(t-1)})$，$h_i=\delta ^2 _{\hat y^{(t-1)}} \cdot l(y_i, \hat y^{(t-1)})$，同时对于第t棵树时，$l(y_i,\hat y _i ^{(t-1)})$为常数。同时去除所有常数项，可以将目标损失函数化成下面的形式：

![image.png](./pic/17.png)

由上面介绍树的复杂度时已经知道：

![image.png](./pic/18.png)

同时可以将目标函数全部转换成在第t棵树所有叶子节点的形式。因为目前对于$\sum_{i=1}^n[g_i f_t(x_i) + h_i f_t(x_i)^2]$可以看作是每个样本在第t棵树的叶子节点得分值相关函数的结果之和，所以也可以将上式从第t棵树的叶子节点上来表示。

![image.png](./pic/19.png)

上式中，前两行$i=[1,n]$求和为在**样本中遍历**，后两行$j=[1,T]$求和为在**叶子节点上遍历**，其中$T$为第t棵树中总叶子节点的个数，$I_j=\{ i|q(x_i)=j \}$表示在第$j$个叶子节点上的样本，$w_j$为第$j$个叶子节点的得分值。

在这里，令$G_j=\sum_{i\in I_j} g_i$，$H_j=\sum_{i\in I_j} h_i$。则：

![image.png](./pic/20.png)

对$w_j$求偏导，并使其导函数等于0，则有：$$G_j + (H_j + \lambda)w_j=0$$

求解得：$w_j^*=-\frac {G_j}{H_j + \lambda}$

目标函数可以化为：$Obj^*=-\frac 1 2 \sum_{j=1}^T \frac {G_j^2}{H_j + \lambda} + \gamma T$

有了新的目标函数后怎么分裂样本数据呢：

![image.png](./pic/21.png)

![image.png](./pic/22.png)

我们所需要做的就是从左到右扫描式尝试所有情况，通过观察Gain的情况来决定最佳该特征的最佳分割点。然后再从所有特征中选择Gain最高的特征作为分割节点。

**如果Gain全都为负怎么办？大家思考一下。**

实际更新y值时并不是将新的树的值直接加上去，而是乘以一个$\eta$来对新树的值进行缩减，通常设置在0.1左右。

## 代码演示

### 数据处理（蛋白质）

In [2]:
import numpy as np

x_dict = {}
x_string = 'ARNDCQEGHILKMFPSTWYV'
for i in range(len(x_string)):
    key = x_string[i]
    x_vector = np.zeros(20)
    x_vector[i] = 1
    x_dict[key] = list(x_vector)
    
y_dict = {1:1, -1:0}

In [5]:
from tkinter import _flatten

def read_file(file):
    data_x = []
    data_y = []
    with open(file) as f:
        for line in f.readlines():
            x, y = line.strip().split(',')
            x_list = [x_dict[i] for i in x]
            x_flatten_list = list(_flatten(x_list))
            data_x.append(x_flatten_list)
            data_y.append(int(y))
            
    return data_x, data_y

In [6]:
file_765 = './newHIV-1_data/746Data.txt'
file_1625 = './newHIV-1_data/1625Data.txt'
file_impens = './newHIV-1_data/impensData.txt'
file_schilling = './newHIV-1_data/schillingData.txt'

data_765_x, data_765_y = read_file(file_765)
data_1625_x, data_1625_y = read_file(file_1625)
data_impens_x, data_impens_y = read_file(file_impens)
data_schilling_x, data_schilling_y = read_file(file_schilling)

data_dict_x = {}
data_dict_y = {}
data_dict_x['765'], data_dict_y['765'] = data_765_x, data_765_y
data_dict_x['1625'], data_dict_y['1625'] = data_1625_x, data_1625_y
data_dict_x['impens'], data_dict_y['impens'] = data_impens_x, data_impens_y
data_dict_x['schilling'], data_dict_y['schilling'] = data_schilling_x, data_schilling_y

data_set = ['765', '1625', 'impens', 'schilling']

### XGBoost参数说明

In [9]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import metrics

#### 参数说明
- eta\[default=0.3\]:学习率参数，保证每一棵树对于结果的影响不太大，从而保证模型的效果。更新叶子节点权重时，乘以该系数，避免步长过大。就是上文中的$\eta$。

- min_child_weight:决定最小叶子节点样本权重和，实际上就是上文中的 $H$ 的阈值。这个参数用于避免过拟合，当它的值较大时，可以避免模型学习到局部的特殊样本。举个例子来说，对正负样本不均衡时的 0-1 分类而言，假设 $h$ 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本，实际是通过控制样本数来控制过拟合的。这个参数设置得越小越容易过拟合，需要通过cv进行调整优化。

- gamma:在树的叶子节点上作进一步分区所需的最小损失减少，即上文中的 $\gamma$。该值越大算法越保守。

- scale_pos_weight:控制样本均衡与否。默认值为1，设置一个正数可以保证快速收敛。

In [22]:
def xgboost_simple(training_set, dataset, data_dict_x, data_dict_y):
    testing_set = dataset[:]
    testing_set.remove(training_set)
    x_train, y_train = data_dict_x[training_set], data_dict_y[training_set]
    x_test, y_test = data_dict_x[testing_set[0]], data_dict_y[testing_set[0]]
    for i in range(2):
        x_test = x_test + data_dict_x[testing_set[i+1]]
        y_test = y_test + data_dict_y[testing_set[i+1]]
    
    xgb = XGBClassifier(learning_rate=0.1, 
                    n_estimators=600, 
                    max_depth=5, 
                    min_child_weight=1, 
                    gamma=0, 
                    subsample=0.8, 
                    colsample_bytree=0.8, 
                    objective='binary:logistic', 
                    nthread=4, 
                    scale_pos_weight=1, 
                    seed=27)

    xgb.fit(np.array(x_train), y_train)
    
    y_pred = xgb.predict(x_test)
    report = metrics.classification_report(y_test, y_pred)

    auc = metrics.roc_auc_score(y_test, y_pred)
    
    print("report: ", report)
    print("auc: ", auc)

### 分别以一个数据集作为训练集并在其他三个数据集上做测试

In [18]:
training_set = '765'
xgboost_simple(training_set, data_set, data_dict_x, data_dict_y)

report:                precision    recall  f1-score   support

          -1       0.94      0.91      0.93      4886
           1       0.62      0.72      0.67       958

   micro avg       0.88      0.88      0.88      5844
   macro avg       0.78      0.82      0.80      5844
weighted avg       0.89      0.88      0.89      5844

auc:  0.8177695721318718


In [19]:
training_set = '1625'
xgboost_simple(training_set, data_set, data_dict_x, data_dict_y)

report:                precision    recall  f1-score   support

          -1       0.90      0.98      0.94      3980
           1       0.86      0.55      0.67       985

   micro avg       0.89      0.89      0.89      4965
   macro avg       0.88      0.76      0.80      4965
weighted avg       0.89      0.89      0.88      4965

auc:  0.7649612274570824


In [20]:
training_set = 'impens'
xgboost_simple(training_set, data_set, data_dict_x, data_dict_y)

report:                precision    recall  f1-score   support

          -1       0.91      0.92      0.91      4432
           1       0.68      0.65      0.66      1211

   micro avg       0.86      0.86      0.86      5643
   macro avg       0.79      0.78      0.79      5643
weighted avg       0.86      0.86      0.86      5643

auc:  0.7821064132336852


In [21]:
training_set = 'schilling'
xgboost_simple(training_set, data_set, data_dict_x, data_dict_y)

report:                precision    recall  f1-score   support

          -1       0.85      0.96      0.90      2392
           1       0.86      0.57      0.68       926

   micro avg       0.85      0.85      0.85      3318
   macro avg       0.85      0.76      0.79      3318
weighted avg       0.85      0.85      0.84      3318

auc:  0.7647517462817022
